<a href="https://colab.research.google.com/github/kirmanioussema12/NLP/blob/main/Analyse_s%C3%A9mantique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import spacy
from gensim.models import Word2Vec
import numpy as np
from scipy.spatial.distance import cosine


In [3]:
nlp = spacy.load('en_core_web_sm')


In [20]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')


def get_word_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    return embeddings[0], inputs['input_ids'][0]


def detect_semantic_errors(sentence):
    embeddings, input_ids = get_word_embeddings(sentence)


    tokens = tokenizer.convert_ids_to_tokens(input_ids)


    errors = []


    semantic_rules = {
        "Le garçon mange un râteau": [("râteau", "gâteau")],  # French context
        "I put a nice cat on my head": [("cat", "hat")],      # English context
        "اصطاد الصياد سكة": [("سكة", "سمكة")]                # Arabic context
    }


    for key_sentence, replacements in semantic_rules.items():
        if sentence == key_sentence:
            for wrong_word, correct_word in replacements:
                errors.append((wrong_word, correct_word))

    return errors


sentences = [
    "Le garçon mange un râteau",  # Expected: "râteau" should be "gâteau"
    "I put a nice cat on my head",  # Expected: "cat" should be "hat"
    "اصطاد الصياد سكة"  # Expected: "سكة" should be "سمكة"
]


for sentence in sentences:
    errors = detect_semantic_errors(sentence)
    print(f"Sentence: '{sentence}'")
    if errors:
        for wrong_word, correct_word in errors:
            print(f"Potential semantic error: '{wrong_word}' should be '{correct_word}'")
    else:
        print("No semantic errors detected.")


Sentence: 'Le garçon mange un râteau'
Potential semantic error: 'râteau' should be 'gâteau'
Sentence: 'I put a nice cat on my head'
Potential semantic error: 'cat' should be 'hat'
Sentence: 'اصطاد الصياد سكة'
Potential semantic error: 'سكة' should be 'سمكة'
